# M1DGMM code lauching to reproduce the results of the experiment in the paper

This notebook presents the data preprocessing used in the paper to benchmark the M1DGMM performance. It is actually a synthesis of the scripts contained in the files named "test_on_\<dataset_name.py\>" in the M1DGMM repository. 
The other models can be run in much the same way using the "test_on_\<dataset_name.py\>" files of the other repositories.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/

/content/gdrive


In [3]:
%cd My Drive

/content/gdrive/My Drive


In [4]:
%cd MDGMM_suite

/content/gdrive/My Drive/MDGMM_suite


In [ ]:
! git clone https://github.com/RobeeF/M1DGMM

In [6]:
!pip install gower

  Created wheel for gower: filename=gower-0.0.5-cp36-none-any.whl size=4233 sha256=b262ca9c00e47f4f66f2b237dc440fb65f87750218a36dbaf0623620a5d25ca7
  Stored in directory: /root/.cache/pip/wheels/c0/09/9b/072d54d6ced0f43a179852e3f09532d0131e25ff7cb4e5ee75
Successfully built gower


In [7]:
!pip install factor-analyzer

     |████████████████████████████████| 40kB 3.6MB/s 
  Created wheel for factor-analyzer: filename=factor_analyzer-0.3.2-cp36-none-any.whl size=40380 sha256=f9c8838f85bcf3999922b7a44e97a6ad3b05ef91cc0b7fb9914bc1652d3a9746
  Stored in directory: /root/.cache/pip/wheels/4a/d0/57/f1330cb9c80e82d8d05391c74c94ed61ce3f03bf6157f3d6db
Successfully built factor-analyzer


In [8]:
!pip install prince

In [9]:
!pip install numdifftools

     |████████████████████████████████| 962kB 5.4MB/s 


In [10]:
%pwd

'/content/gdrive/My Drive/MDGMM_suite'

In [11]:
import os 

os.chdir('/content/gdrive/My Drive/MDGMM_suite/M1DGMM')

from copy import deepcopy

from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder

from gower import gower_matrix
from sklearn.metrics import silhouette_score


import pandas as pd
import matplotlib.pyplot as plt
from m1dgmm import M1DGMM
from init_params import dim_reduce_init
from metrics import misc, cluster_purity
from data_preprocessing import gen_categ_as_bin_dataset, \
        compute_nj

import autograd.numpy as np
from numpy.random import uniform


# M1GMM benchmarking on Australian Credit

In [12]:
###############################################################################
################################ Credit data    ###############################
###############################################################################

#===========================================#
# Importing data
#===========================================#
os.chdir('/content/gdrive/My Drive/MDGMM_suite/datasets')

credit = pd.read_csv('australian_credit/australian.csv', sep = ' ', header = None)
y = credit.iloc[:,:-1]
labels = credit.iloc[:,-1]

y = y.infer_objects()
numobs = len(y)


n_clusters = len(np.unique(labels))
p = y.shape[1]

#===========================================#
# Formating the data
#===========================================#
var_distrib = np.array(['bernoulli', 'continuous', 'continuous', 'categorical',\
                        'categorical', 'categorical', 'continuous', 'bernoulli',\
                        'bernoulli', 'continuous', 'bernoulli', 'categorical',\
                        'continuous', 'continuous']) 
 
# No ordinal data 
 
y_categ_non_enc = deepcopy(y)
vd_categ_non_enc = deepcopy(var_distrib)

# Encode categorical datas
# Test to encode categorical variables
le = LabelEncoder()
for col_idx, colname in enumerate(y.columns):
    if var_distrib[col_idx] == 'categorical': 
        y[colname] = le.fit_transform(y[colname])

# No binary data 

enc = OneHotEncoder(sparse = False, drop = 'first')
labels_oh = enc.fit_transform(np.array(labels).reshape(-1,1)).flatten()

nj, nj_bin, nj_ord, nj_categ = compute_nj(y, var_distrib)
y_np = y.values
nb_cont = np.sum(var_distrib == 'continuous')

p_new = y.shape[1]

# Feature category (cf)
cf_non_enc = np.logical_or(vd_categ_non_enc == 'categorical', vd_categ_non_enc == 'bernoulli')

# Non encoded version of the dataset:
y_nenc_typed = y_categ_non_enc.astype(np.object)
y_np_nenc = y_nenc_typed.values

# Defining distances over the non encoded features
dm = gower_matrix(y_nenc_typed, cat_features = cf_non_enc) 

dtype = {y.columns[j]: np.float64 if (var_distrib[j] != 'bernoulli') and \
        (var_distrib[j] != 'categorical') else np.str for j in range(p_new)}

y = y.astype(dtype, copy=True)



In [ ]:
import matplotlib as mpl

backend_ =  mpl.get_backend() 
mpl.use("Agg")  # Prevents from showing the plots to gain space

# First: automatically find the best architecture
r = np.array([5, 4, 3])
numobs = len(y)
k = [4, n_clusters]
eps = 1E-05
it = 3
maxstep = 100

prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None, use_famd = True)
out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it, eps, maxstep, seed = None)


# Then run the best architecture 30 times

r = out['best_r']
numobs = len(y)
k = out['best_k']
eps = 1E-05
it = 30
maxstep = 100

nb_trials= 30
m1dgmm_res = pd.DataFrame(columns = ['it_id', 'micro', 'macro', 'silhouette'])

for i in range(nb_trials):
    prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None,\
                                  use_famd = True)

    out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it,\
                  eps, maxstep, seed = None, perform_selec = False)
    m, pred = misc(labels_oh, out['classes'], True) 
    plt.close()

    sil = silhouette_score(dm, pred, metric = 'precomputed')
    micro = precision_score(labels_oh, pred, average = 'micro')
    macro = precision_score(labels_oh, pred, average = 'macro')

    m1dgmm_res = m1dgmm_res.append({'it_id': i + 1, 'micro': micro, 'macro': macro, \
                                'silhouette': sil}, ignore_index=True)

  
mpl.use(backend_) # Reset backend

In [14]:
print(m1dgmm_res.mean())
print(m1dgmm_res.std())

it_id         15.500000
micro          0.698696
macro          0.802738
silhouette     0.176574
dtype: float64
it_id         8.803408
micro         0.117355
macro         0.045442
silhouette    0.029559
dtype: float64


# Heart dataset

In [15]:
%cd ../datasets

/content/gdrive/My Drive/MDGMM_suite


In [16]:
#===========================================#
# Importing data
#===========================================#

heart = pd.read_csv('heart_statlog/heart.csv', sep = ' ', header = None)
y = heart.iloc[:,:-1]
labels = heart.iloc[:,-1]
labels = np.where(labels == 1, 0, labels)
labels = np.where(labels == 2, 1, labels)

y = y.infer_objects()
numobs = len(y)

# Too many zeros for this "continuous variable". Add a little noise to avoid 
# the correlation matrix for each group to blow up
uniform_draws = uniform(0, 1E-12, numobs)
y.iloc[:, 9] = np.where(y[9] == 0, uniform_draws, y[9])

n_clusters = len(np.unique(labels))
p = y.shape[1]

#===========================================#
# Formating the data
#===========================================#
var_distrib = np.array(['continuous', 'bernoulli', 'categorical', 'continuous',\
                        'continuous', 'bernoulli', 'categorical', 'continuous',\
                        'bernoulli', 'continuous', 'ordinal', 'ordinal',\
                        'categorical']) # Last one is ordinal for me (but real
                        # real in the data description)
    
# Ordinal data already encoded
 
y_categ_non_enc = deepcopy(y)
vd_categ_non_enc = deepcopy(var_distrib)

# Encode categorical datas
#y, var_distrib = gen_categ_as_bin_dataset(y, var_distrib)

#######################################################
# Test to encode categorical variables
le = LabelEncoder()
for col_idx, colname in enumerate(y.columns):
    if var_distrib[col_idx] == 'categorical': 
        y[colname] = le.fit_transform(y[colname])

#################################################

# Encode binary data
le = LabelEncoder()
for col_idx, colname in enumerate(y.columns):
    if var_distrib[col_idx] == 'bernoulli': 
        y[colname] = le.fit_transform(y[colname])
    
enc = OneHotEncoder(sparse = False, drop = 'first')
labels_oh = enc.fit_transform(np.array(labels).reshape(-1,1)).flatten()

nj, nj_bin, nj_ord, nj_categ = compute_nj(y, var_distrib)
y_np = y.values
nb_cont = np.sum(var_distrib == 'continuous')

p_new = y.shape[1]


# Feature category (cf)
cf_non_enc = np.logical_or(vd_categ_non_enc == 'categorical', vd_categ_non_enc == 'bernoulli')

# Non encoded version of the dataset:
y_nenc_typed = y_categ_non_enc.astype(np.object)
y_np_nenc = y_nenc_typed.values

# Defining distances over the non encoded features
dm = gower_matrix(y_nenc_typed, cat_features = cf_non_enc) 

dtype = {y.columns[j]: np.float64 if (var_distrib[j] != 'bernoulli') and \
        (var_distrib[j] != 'categorical') else np.str for j in range(p_new)}

y = y.astype(dtype, copy=True)

In [ ]:
# MDGMM. 
import matplotlib as mpl

backend_ =  mpl.get_backend() 
mpl.use("Agg")  # Prevents from showing the plots to gain space

# First: automatically find the best architecture 
r = np.array([5, 4, 3])
numobs = len(y)
k = [4, n_clusters]
eps = 1E-05
it = 3
maxstep = 100

prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None, use_famd = True)
out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it, eps, maxstep, seed = None)


# Then run the best architecture 30 times
r = out['best_r']
numobs = len(y)
k = out['best_k']
eps = 1E-05
it = 30
maxstep = 100


nb_trials= 30
m1dgmm_res = pd.DataFrame(columns = ['it_id', 'micro', 'macro', 'silhouette'])

for i in range(nb_trials):
    prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None,\
                                  use_famd = True)

    out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it,\
                  eps, maxstep, seed = None, perform_selec = False)
    m, pred = misc(labels_oh, out['classes'], True) 
    try:
      sil = silhouette_score(dm, pred, metric = 'precomputed')
    except:
      sil = -1
    micro = precision_score(labels_oh, pred, average = 'micro')
    macro = precision_score(labels_oh, pred, average = 'macro')
    print(micro)
    print(macro)

    m1dgmm_res = m1dgmm_res.append({'it_id': i + 1, 'micro': micro, 'macro': macro, \
                                'silhouette': sil}, ignore_index=True)


mpl.use(backend_) # Reset backend

In [18]:
print(m1dgmm_res.mean())
print(m1dgmm_res.std())

it_id         15.500000
micro          0.825185
macro          0.824007
silhouette     0.254065
dtype: float64
it_id         8.803408
micro         0.011928
macro         0.012497
silhouette    0.002808
dtype: float64


## PIMA

In [19]:
os.chdir('/content/gdrive/My Drive/MDGMM_suite/datasets')

In [20]:
pima = pd.read_csv('pima/pima_indians.csv', sep = ',')
y = pima.iloc[:,:-1]
labels = pima.iloc[:,-1]

y = y.infer_objects()
numobs = len(y)


n_clusters = len(np.unique(labels))
p = y.shape[1]

#===========================================#
# Formating the data
#===========================================#
var_distrib = np.array(['ordinal', 'continuous', 'continuous', 'continuous',\
                        'continuous', 'continuous', 'continuous', 'continuous']) 
 
# Ordinal data already encoded
 
y_categ_non_enc = deepcopy(y)
vd_categ_non_enc = deepcopy(var_distrib)

# No categ data
# No binary data 

enc = OneHotEncoder(sparse = False, drop = 'first')
labels_oh = enc.fit_transform(np.array(labels).reshape(-1,1)).flatten()

nj, nj_bin, nj_ord, n_categ = compute_nj(y, var_distrib)
y_np = y.values
nb_cont = np.sum(var_distrib == 'continuous')

p_new = y.shape[1]

# Feature category (cf)
cf_non_enc = np.logical_or(vd_categ_non_enc == 'categorical', vd_categ_non_enc == 'bernoulli')

# Non encoded version of the dataset:
y_nenc_typed = y_categ_non_enc.astype(np.object)
y_np_nenc = y_nenc_typed.values

# Defining distances over the non encoded features
dm = gower_matrix(y_nenc_typed, cat_features = cf_non_enc) 


dtype = {y.columns[j]: np.float64 if (var_distrib[j] != 'bernoulli') and \
        (var_distrib[j] != 'categorical') else np.str for j in range(p_new)}
dtype['Pregnancies'] = np.str

y = y.astype(dtype, copy=True)

In [ ]:
import matplotlib as mpl

backend_ =  mpl.get_backend() 
mpl.use("Agg")  # Prevents from showing the plots to gain space

# First: automatically find the best architecture 
r = np.array([5, 4, 3])
numobs = len(y)
k = [4, n_clusters]
eps = 1E-05
it = 2
maxstep = 100

prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None,\
                              use_famd = True)
out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it, eps,\
             maxstep, seed = None)

# Then run the best architecture 30 times
r = out['best_r']
numobs = len(y)
k = out['best_k']
eps = 1E-05
it = 30
maxstep = 100

nb_trials= 30
m1dgmm_res = pd.DataFrame(columns = ['it_id', 'micro', 'macro', 'silhouette'])

for i in range(nb_trials):
    prince_init = dim_reduce_init(y, n_clusters, k, r, nj, var_distrib, seed = None,\
                                  use_famd = True)
    
    out = M1DGMM(y_np, n_clusters, r, k, prince_init, var_distrib, nj, it,\
                  eps, maxstep, seed = None, perform_selec = False)
    m, pred = misc(labels_oh, out['classes'], True) 

    try:
      sil = silhouette_score(dm, pred, metric = 'precomputed')
    except:
      sil = -1
    micro = precision_score(labels_oh, pred, average = 'micro')
    macro = precision_score(labels_oh, pred, average = 'macro')

    m1dgmm_res = m1dgmm_res.append({'it_id': i + 1, 'micro': micro, 'macro': macro, \
                                'silhouette': sil}, ignore_index=True)


mpl.use(backend_) # Reset backend



In [22]:
print(m1dgmm_res.mean())
print(m1dgmm_res.std())

it_id         15.500000
micro          0.641189
macro          0.616231
silhouette     0.230101
dtype: float64
it_id         8.803408
micro         0.024794
macro         0.020724
silhouette    0.016014
dtype: float64
